In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from meteostat import Point, Daily
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

In [2]:
df_filtered = pd.read_csv('../buffelgrass-onetime-test.csv')
df_filtered.head()

,Observation_ID,Observation_Date,Create_Date,Latitude,Longitude,Abundance_Name,Abundance_Binary
0,3152,7/5/2021 7:00,7/5/2021,32.367762,-111.113910,NaN,0
1,3074,9/15/2020 7:00,9/18/2020,32.270887,-111.244599,95% or more,1
2,2177,8/18/2019 7:00,8/18/2019,36.907189,-110.623412,95% or more,1
3,3064,9/9/2020 7:00,9/9/2020,32.209717,-110.791179,50-74%,1
4,2150,7/9/2019 7:00,7/9/2019,32.194713,-110.914275,NaN,0


In [3]:
scale = 4000 ## meters
label = df_filtered.Abundance_Binary.values ## labels
for days in range(24):
    
    pred = []
    
    for i in tqdm(range(len(df_filtered))):
            
        ## filter date
        start_date = df_filtered.Observation_Date.values[i].split()[0]
        start_month, start_day, start_year = start_date.split('/')
        start_month = int(start_month)
        start_day = int(start_day)
        start_year = int(start_year)
        start = datetime(start_year, start_month, start_day)
        
        end_date = (datetime.strptime(start_date, "%m/%d/%Y") + timedelta(days=days)).strftime("%m/%d/%Y")
        end_month, end_day, end_year = end_date.split('/')
        end_month = int(end_month)
        end_day = int(end_day)
        end_year = int(end_year)
        end = datetime(end_year, end_month, end_day)
        
        ## point location
        lat = df_filtered.Latitude.values[i]
        long = df_filtered.Longitude.values[i]
        #elevation = df_filtered.Elevation_in_Meters.values[i]
        #point = Point(lat, long, elevation)
        point = Point(lat, long)
        data = Daily(point, start, end)
        data = data.fetch()
        
        ## get precipitation time series data
        try:
            ts_prec = data['prcp'].values*0.0393701 ## 1mm = 0.0393701 inches        
            pred.append(1*(sum(ts_prec)>1.7))
        except:
            pred.append(0)
    
    tn, fp, fn, tp = confusion_matrix(pred, label).ravel()            
    acc = 100*(tn+tp)/len(label)
    fp_rate = 100*(fp)/len(label)
    fn_rate = 100*(fn)/len(label)
    
    print(f'Accumulative Days {days}.')
    print(f'Accuracy: {acc}%')
    print(f'FP: {fp_rate}%')
    print(f'FN: {fn_rate}%')

100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 50.18it/s]


Accumulative Days 0.
Accuracy: 51.282051282051285%
FP: 48.717948717948715%
FN: 0.0%


100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 48.62it/s]


Accumulative Days 1.
Accuracy: 51.282051282051285%
FP: 48.717948717948715%
FN: 0.0%


100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 51.78it/s]


...

Accumulative Days 21.
Accuracy: 43.58974358974359%
FP: 41.02564102564103%
FN: 15.384615384615385%


100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 49.75it/s]


Accumulative Days 22.
Accuracy: 46.15384615384615%
FP: 38.46153846153846%
FN: 15.384615384615385%


100%|██████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 50.57it/s]

Accumulative Days 23.
Accuracy: 46.15384615384615%
FP: 38.46153846153846%
FN: 15.384615384615385%
